# Exercises 02
### Number representation

1\. Write a function that converts number representation, bin<->dec<->hex. (Clearly using the corresponding python built-in functions is not fair..)

In [1]:
def conversion(number, type_conv: [str, str]):
    
    if type_conv[0] == type_conv[1]:
        return number
    
    elif type_conv[0] == 'bin':
        number_conv = str(number)
        sum = 0
        
        if type_conv[1] == 'dec':
            for i in range(len(number_conv)):
                sum += int(number_conv[i]) * (2 ** (len(number_conv) - i - 1))
            return sum
            
        elif type_conv[1] == 'hex':
            decimal_num = conversion(number, ['bin', 'dec'])
            hexadecimal_digits = "0123456789ABCDEF"
            result = []
            
            while decimal_num > 0:
                remainder = decimal_num % 16
                result.append(hexadecimal_digits[remainder])
                decimal_num = decimal_num // 16
                result.reverse()
        return ''.join(result) if result else '0'
    
    elif type_conv[0] == 'dec':
        if type_conv[1] == 'bin':
            binary_result = ''
            decimal = int(number)
            
            while decimal > 0:
                remainder = decimal % 2
                binary_result = str(remainder) + binary_result
                decimal = decimal // 2
            return binary_result if binary_result else '0'
        
        elif type_conv[1] == 'hex':
            return conversion(conversion(number, ['dec', 'bin']), ['bin', 'hex'])
        
    elif type_conv[0] == 'hex':
        hex_to_bin_dict = {
        '0': '0000', '1': '0001', '2': '0010', '3': '0011',
        '4': '0100', '5': '0101', '6': '0110', '7': '0111',
        '8': '1000', '9': '1001', 'A': '1010', 'B': '1011',
        'C': '1100', 'D': '1101', 'E': '1110', 'F': '1111'
        }
        
        if type_conv[1] == 'bin':
            binary_result = ''
            for hex_digit in number:
                binary_result += hex_to_bin_dict[hex_digit]
            return binary_result
        if type_conv[1] == 'dec':
            return conversion(conversion(number, ['hex', 'bin']), ['bin', 'dec'])

In [2]:
print(conversion('00111100', ['bin', 'dec']))
print(conversion('60', ['dec', 'hex']))
print(conversion('3C', ['hex', 'bin']))

60
3C
00111100


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [33]:
def floating_creation(x: str = []):
    # Extracting sign bit
    sign = int(x[0])

    # Extracting exponent bits and converting from binary to decimal
    exp = int(conversion(x[1:9], ['bin', 'dec']))

    # Extracting mantissa bits and converting from binary to decimal
    mantissa_bin = x[9:]
    mantissa_dec = 1 + sum(int(b) * 2**(-i-1) for i, b in enumerate(mantissa_bin))

    # Calculating the result using the formula for floating-point representation
    result = ((-1) ** sign) * mantissa_dec * 2 ** (exp - 127)

    return result

In [34]:
floating_creation('01000000110011000000000000000000')

6.375

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits

In [37]:
def find_underflow_limit():
    underflow_limit = 1.0
    while not underflow_limit / 2 == 0.0:
        underflow_limit /= 2
    return underflow_limit

def find_overflow_limit():
    overflow_limit = 1.0
    while not overflow_limit * 2 == float('inf'):
        overflow_limit *= 2
    return overflow_limit

In [39]:
print(find_overflow_limit())
print(find_underflow_limit())

8.98846567431158e+307
5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [40]:
def find_machine_precision():
    x = 10.0
    z = 1/2
    
    while not x == x + z:
        z /= 2
    return z

In [41]:
print(find_machine_precision())

8.881784197001252e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [3]:
def quadratic_formula(a, b, c):
    result = [0, 0]
    result[0] = (-b + (b ** 2 - 4 * a * c) ** (1/2))/(2 * a)
    result[1] = (-b - (b ** 2 - 4 * a * c) ** (1/2))/(2 * a)
    
    return result

quadratic_formula(a = 0.001, b = 1000, c = 0.001)

[-9.999894245993346e-07, -999999.999999]

In [9]:
def quadratic_formula(a, b, c):
    result = [0, 0]
    result[0] = (4 * a * c)/((2 * a) * (-b - (b ** 2 - 4 * a * c) ** (1/2)))
    result[1] = (4 * a * c)/((2 * a) * (-b + (b ** 2 - 4 * a * c) ** (1/2)))
    
    return result

quadratic_formula(a = 0.001, b = 1000, c = 0.001)

[-1.0000000000009999e-06, -1000010.5755125057]

We see a difference due to numerical instability: this is the consequence of the fact that the coefficient $a$ is really close to 0. Thus, the enhanced version of our quadratica formula must be able to compute also the cases in which $a$ is close to 0 without changing the result.

In [16]:
def quadratic_solver_enhanced(a, b, c):
    # Handle the case when a is close to zero
    if abs(a) < 1e-10:
        # Handle the case when b is close to zero
        if abs(b) < 1e-10:
            return "Infinite solutions"
        else:
            x = -c / b
            return x
    else:
        delta = b**2 - 4*a*c
        if delta < 0:
            return "No real roots"
        else:
            x1 = (-b + delta ** (1/2)) / (2*a)
            x2 = (-b - delta ** (1/2)) / (2*a)
            return x1, x2

print(quadratic_solver_enhanced(a = 0.001, b = 1000, c = 0.001))

(-9.999894245993346e-07, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [21]:
def func(x):
    return x * (x - 1)

def derivative(function, x, d):
    der = (function(x + d) - function(x)) / d
    return der

derivative(func, x = 1, d = 1e-2)

1.010000000000001

We can see that the result is not perfect (it should be exactly $1$); this is due to the fact that the limit $d$ -> 0 is not respected, $d$ is not small enough in order to obtain the correct derivative.

In [23]:
print(derivative(func, x = 1, d = 1e-4))
print(derivative(func, x = 1, d = 1e-6))
print(derivative(func, x = 1, d = 1e-8))
print(derivative(func, x = 1, d = 1e-10))
print(derivative(func, x = 1, d = 1e-12))
print(derivative(func, x = 1, d = 1e-14))

1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509


We see that the most accurate solution is with $d = 10^{-8}$; with smaller value for $d$ the error grows. This is probably due to the fact that my computer loses precision for smaller floating numbers.

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [78]:
def semicircle(x):
    return ((1 - x ** 2) ** (1/2))

def integral(function, N = int(10), limits = []):
    slice_width = (limits[1] - limits[0])/N
    sum = 0
    for i in range(int(N)):
        sum += slice_width * function(limits[0] + i * slice_width)
        
    return sum

print(integral(semicircle, N= 100, limits= [-1, 1]))

1.5691342555492505


The result is a little bit off, but increasing the size of $N$ it will increase the accuracy.

In [66]:
import time

def time_spent(N):
    start_time = time.time()
    integral(semicircle, N, limits= [-1, 1])
    end_time = time.time()
    return end_time - start_time

N = 1000000
while time_spent(N) < 1:
    N += 100000

N -= 100000

while time_spent(N) < 1:
    N += 10000

N -= 10000
    
while time_spent(N) < 1:
    N += 1000

N -= 1000
  
while time_spent(N) < 1:
    N += 100

N -= 100

while time_spent(N) < 1:
    N += 10

N -= 10

while time_spent(N) < 1:
    N += 1

print(N)
print(integral(semicircle, N, limits= [-1, 1]))


4238890
1.570796326604501


In [86]:
%time integral(semicircle, N= 200000000, limits= [-1, 1])

CPU times: user 47.2 s, sys: 0 ns, total: 47.2 s
Wall time: 47.2 s


1.5707963267944416